In [1]:
import torch

In [2]:
from comet_ml import Experiment, Optimizer

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import torch
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

/tmp/ipykernel_1197867/655870062.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [4]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [5]:
from hypertab import HypernetworkPCA, TrainingModes, Hypernetwork
from hypertab.interfaces import HypernetworkSklearnInterface
# from ipynb.fs.defs.MNIST_benchmark import test_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [6]:
from sklearn.metrics import classification_report, balanced_accuracy_score, accuracy_score

In [7]:
import pyhopper
import sklearn.datasets

/home/MCB/wwydmanski/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [9]:
DEVICE="cuda:0"

In [10]:
GS_METRIC = 'balanced_accuracy'

# Load data

In [11]:
TEST_RUN = False

# Preprocess

In [12]:
def train_test_split_tuple(X, y, train_size=None):
    if isinstance(X, tuple) and isinstance(y, tuple):
        X_train, X_test = X
        y_train, y_test = y
    else:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, stratify=y)
    
    return X_train, X_test, y_train, y_test

In [13]:
import numpy as np
np.random.seed(42)

def prepare_data(X, y, size=None):
    if isinstance(X, tuple) and isinstance(y, tuple):
        X_train, X_test = X
        y_train, y_test = y
    else:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, stratify=y)
    # X_train, y_train = imblearn.over_sampling.RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train, X_test, y_train, y_test = [torch.from_numpy(x) for x in [X_train, X_test, y_train, y_test]]
    
    return X_train, X_test, y_train, y_test

## Benchmark

In [14]:
criterion = torch.nn.CrossEntropyLoss()

In [15]:
def _summarize_results(y_pred, y_score, y_test, labels):
    results = []
    for idx, label in enumerate(labels):
        y_pred_filt = y_pred[y_test==idx]
        y_test_filt = y_test[y_test==idx]
#         acc = (y_pred_filt==y_test_filt.numpy()).sum()/len(y_test_filt)*100
        acc = accuracy_score(y_test_filt, y_pred_filt)
        results.append({
            "Class": label,
            "Metric": acc
        })
        
#     acc = (y_pred==y_test.numpy()).sum()/len(y_test)*100 
    acc = accuracy_score(y_test, y_pred)
    results.append({
        "Class": "Total",
        "Metric": acc
    })
    
    results.append({
        "Class": "balanced_accuracy",
        "Metric": balanced_accuracy_score(y_test, torch.from_numpy(y_pred)).item()*100
    })
    
    try:
        results.append({
            "Class": "F1 score",
            "Metric": f1_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "roc_auc",
            "Metric": roc_auc_score(y_test, torch.from_numpy(y_score[:, 1])).item()*100
        })
        results.append({
            "Class": "Precision",
            "Metric": precision_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "Recall",
            "Metric": recall_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
    except ValueError:
        pass
    return results

def test_model(model_fn, data, train_size, label_encoder=None, iters=10, as_numpy=False):
    if TEST_RUN:
        iters = 1
        
    if label_encoder is not None:
        labels = label_encoder.classes_
    else:
        labels = sorted(pd.unique(data[1][0] if isinstance(data[1], tuple) else data[1]))

    
    results = []
    
    X, y = data

    for i in range(iters):
        X_train, X_test, y_train, y_test = prepare_data(*data, train_size)
        print('iter', i+1, 'of', iters, 'X_train shape', X_train.shape)

        model = model_fn()

        if as_numpy:
            model.fit(X_train.numpy(), y_train.numpy());
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        results.extend(_summarize_results(y_pred, y_score, y_test, labels))

    dframe = pd.DataFrame.from_dict(results)
    # sns.violinplot(data=dframe[dframe["Class"]!="Loss"], y="Class", x="Metric", orient='h')
    return dframe

### Param search

In [16]:
def pyhopper_best_params(model_fn, param_grid, data, train_size, metric=GS_METRIC, time="30m", default_params={}):
    if TEST_RUN:
        time = 60
        if 'epochs' in param_grid:
            param_grid["epochs"] = pyhopper.choice([10])
    
    X, y = data
    print('pyhopper', X.shape, y.shape, train_size)
        
    def objective(params):
    #     print("Training...")
        print('params',params)
        model_results = test_model(
                        model_fn(
                            **default_params,
                            **params
                        ),
                        (X, y),
                        train_size,
                        None, 5)
        with open(f"params/{DATA}_{model_fn.__name__}_params.txt", "a") as f:
            f.write(str(params) + ", " + str(model_results[model_results["Class"]==metric]["Metric"].mean()) + "\n")
        return model_results[model_results["Class"]==metric]["Metric"].mean()

    from pyhopper.callbacks import History
    search = pyhopper.Search(param_grid)

    best_params = search.run(objective, "maximize", time, n_jobs=1, seeding_ratio=0.5)
    
    with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
            f.write(str(best_params))
    
    print(f"{DATA}_{model_fn.__name__}_{best_params}")
    return best_params


In [17]:
d = {}

In [18]:
DATA = "Connectionist"

# TRAIN

In [19]:
import sklearn
import sklearn.datasets

if DATA == "BreastCancer":
    dataset = sklearn.datasets.load_breast_cancer()
    X = dataset['data']
    y = dataset['target']
elif DATA == "Connectionist":
    dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header=None)
    X = dataset.values[:, :-1].astype(float)
    y = dataset.values[:, -1]
    y = LabelEncoder().fit_transform(y)
elif DATA == "Dermatology":
    dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data", header=None, na_values="?").dropna()
    X = dataset.values[:, :-1].astype(float)
    y = dataset.values[:, -1].astype(int) - 1
elif DATA == "Glass":
    dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data", header=None, na_values="?").dropna()
    X = dataset.values[:, :-1].astype(float)
    y = dataset.values[:, -1].astype(int)
    y = LabelEncoder().fit_transform(y).astype(int)
    
elif DATA == "Cleveland":
    dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", header=None, na_values="?").dropna()
    X = dataset.values[:, :-1].astype(float)
    y = dataset.values[:, -1].astype(int)
    y = LabelEncoder().fit_transform(y).astype(int)

elif DATA == "CNAE9":
    dataset = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00233/CNAE-9.data", header=None, na_values="?").dropna()
    X = dataset.values[:, 1:].astype(float)
    y = dataset.values[:, 0].astype(int)
    y = LabelEncoder().fit_transform(y).astype(int)

max_size = int(len(X)*0.7)
print(X.shape, len(np.unique(y)), max_size)

(208, 60) 2 145


In [20]:
n_classes = len(np.unique(y))
n_features = X.shape[1]
unique, counts = np.unique(y, return_counts=True)

print(dict(zip(unique, counts)))

print('n_classes', n_classes)
print('n_features', n_features)

{0: 111, 1: 97}
n_classes 2
n_features 60


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y, random_state=42)

eval_max_size = int(len(X_train))
train_max_size = int(len(X_train) * 0.75)
print('train_max_size', train_max_size)
print('eval_max_size', eval_max_size)

train_max_size 124
eval_max_size 166


In [22]:
X_train.shape

(166, 60)

## HyperTab

In [23]:
def hypertab_fn(epochs=150, masks_no=100, mask_size=100, target_size=100, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = Hypernetwork(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)],
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_features, 32), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(32, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=3e-4)
        return network
    return _inner

In [ ]:
param_grid = {
    "epochs": pyhopper.choice([100, 150]),
    "masks_no": pyhopper.choice([10, 20, 50, 80, 100, 150, 200]),
    "mask_size": pyhopper.choice([2, 5, 10, 15, 20, 25]),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "batch_size": pyhopper.choice([32, 64]),
}
#{'epochs': 100, 'masks_no': 10, 'mask_size': 2, 'target_size': 5, 'lr': 3e-05, 'batch_size': 32}
hp_best_params = pyhopper_best_params(
    hypertab_fn, param_grid, data=(X_train, y_train), train_size=train_max_size, time="60m"
)
hp_best_params

pyhopper (166, 60) (166,) 124


  0%|                                                                                                                                                       | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
params {'epochs': 100, 'masks_no': 10, 'mask_size': 2, 'target_size': 5, 'batch_size': 32}
iter 1 of 5 X_train shape torch.Size([124, 60])
iter 2 of 5 X_train shape torch.Size([124, 60])
iter 3 of 5 X_train shape torch.Size([124, 60])
iter 4 of 5 X_train shape torch.Size([124, 60])
iter 5 of 5 X_train shape torch.Size([124, 60])


Best f: 76.2 (out of 1 params):   5%|████▋                                                                                               | [02:48<57:10, 0.4 param/min]

params {'epochs': 100, 'masks_no': 50, 'mask_size': 20, 'target_size': 50, 'batch_size': 32}
iter 1 of 5 X_train shape torch.Size([124, 60])


In [57]:
epochs = hp_best_params['epochs']
masks_no = hp_best_params['masks_no']
mask_size = hp_best_params['mask_size']
target_size = hp_best_params['target_size']
lr = 3e-4
batch_size = hp_best_params['batch_size']

model = hypertab_fn(
    epochs, 
    masks_no, 
    mask_size, 
    target_size, 
    lr,
    batch_size=batch_size,
    verbose=True)()

In [58]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_score = model.predict_proba(X_test)
hyper_results = _summarize_results(y_pred, 
                                   y_score, 
                                   torch.from_numpy(y_test), 
                                   torch.from_numpy(np.unique(y)))
hyper_results = pd.DataFrame(hyper_results)



  0%|                                                                                                                                          | 0/150 [00:00<?, ?it/s]

  1%|▊                                                                                                                                 | 1/150 [00:01<03:47,  1.52s/it]

  1%|█▋                                                                                                                                | 2/150 [00:01<02:05,  1.17it/s]

  2%|██▌                                                                                                                               | 3/150 [00:02<01:32,  1.60it/s]

  3%|███▍                                                                                                                              | 4/150 [00:02<01:16,  1.90it/s]

  3%|████▎                                                                                                                             | 5/150 [00:02<01:

In [59]:
res = hyper_results[hyper_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{eval_max_size}: {res.mean():.3f} ~ {res.std():.2f}, (max: {res.max():.3f})")

166: 0.833 ~ nan, (max: 0.833)


In [60]:
with open("results/" + DATA + "_hypertab.txt", "w") as f:
    f.write(f"{eval_max_size}: {res.mean():.3f} ~ {res.std():.2f}, (max: {res.max():.3f})")

## XGBoost

In [26]:
import xgboost
xgboost.__version__

'1.7.4'

In [27]:
def get_xgboost(**params):
    random_seed = np.random.randint(1024)
    def _inner():
        return xgboost.XGBClassifier(
            verbosity=0,
            random_state=random_seed,
            use_label_encoder=False,
            **params
        )
    return _inner    

In [ ]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50, init=50),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbc = get_xgboost()

xgbt_best1 = pyhopper_best_params(get_xgboost, param_grid, data=(X_train, y_train), train_size=train_max_size, time="10m")

pyhopper (166, 60) (166,) 124


  0%|                                                                                                                                                                                                                                                      | [00:00<?]

Search is scheduled for 10:00 (m:s)
params {'n_estimators': 50, 'max_depth': 2, 'learning_rate': 0.0010000000000000002, 'min_child_weight': 1, 'gamma': 0}
iter 1 of 5 X_train shape torch.Size([124, 60])


/home/MCB/wwydmanski/miniconda3/envs/torch/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


iter 2 of 5 X_train shape torch.Size([124, 60])


...

/home/MCB/wwydmanski/miniconda3/envs/torch/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


iter 4 of 5 X_train shape torch.Size([124, 60])


/home/MCB/wwydmanski/miniconda3/envs/torch/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


iter 5 of 5 X_train shape torch.Size([124, 60])


/home/MCB/wwydmanski/miniconda3/envs/torch/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [ ]:
xgbt_best1

In [ ]:
model = get_xgboost(**xgbt_best1)()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_score = model.predict_proba(X_test)
hyper_results = _summarize_results(y_pred, 
                                   y_score, 
                                   torch.from_numpy(y_test), 
                                   torch.from_numpy(np.unique(y)))
hyper_results = pd.DataFrame(hyper_results)

In [ ]:
res = hyper_results[hyper_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{eval_max_size}: {res.mean():.3f} ~ {res.std():.2f}, (max: {res.max():.3f})")

In [ ]:
with open("results/" + DATA + "_xgboost.txt", "w") as f:
    f.write(f"{eval_max_size}: {res.mean():.3f} ~ {res.std():.2f}, (max: {res.max():.3f})")